# Lab-1.7: DPO (Direct Preference Optimization) 訓練實作

**實驗目標**: 實現並訓練 DPO 對齊模型

這個 notebook 將實現 DPO 的核心訓練流程。DPO 通過直接優化偏好數據來改善模型行為，相比傳統 RLHF 更加穩定且高效。

## DPO 核心原理

DPO 的核心想法是直接從偏好數據中學習最優策略，而不需要顯式的獎勵模型：

**DPO 損失函數**:
```
L_DPO = -E[(x,y_w,y_l)~D][log σ(β log π_θ(y_w|x)/π_ref(y_w|x) - β log π_θ(y_l|x)/π_ref(y_l|x))]
```

其中：
- `π_θ`: 正在訓練的策略模型
- `π_ref`: 參考模型 (SFT 模型)
- `β`: 溫度參數，控制偏好強度
- `y_w`: 偏好回應 (chosen)
- `y_l`: 非偏好回應 (rejected)

---

## 步驟 1: 環境準備與導入

載入必要的庫並準備 DPO 訓練環境。

In [ ]:
import torch
import torch.nn.functional as F
import os
import numpy as np
from copy import deepcopy
from pathlib import Path

from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments
)
from peft import PeftModel, LoraConfig, get_peft_model
from datasets import load_from_disk, Dataset
from trl import DPOTrainer

# 設置隨機種子
torch.manual_seed(42)
np.random.seed(42)

print('🚀 開始 DPO 訓練實作')
print(f'GPU 可用: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'當前 GPU: {torch.cuda.get_device_name()}')
    print(f'GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

## 步驟 2: 載入 SFT 基線模型

載入之前訓練的 SFT 模型作為 DPO 的起始點和參考模型。

In [ ]:
# 模型路徑配置
BASE_MODEL_NAME = 'microsoft/DialoGPT-medium'
SFT_MODEL_PATH = './sft_model_output'

print(f'📦 載入 SFT 基線模型...')

# 載入 tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# 載入基礎模型
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    torch_dtype=torch.float16,
    device_map='auto' if torch.cuda.is_available() else 'cpu'
)

print(f'✅ 基礎模型載入成功')

# 檢查 SFT 模型是否存在
if os.path.exists(SFT_MODEL_PATH):
    try:
        # 載入 SFT 模型作為策略模型
        policy_model = PeftModel.from_pretrained(base_model, SFT_MODEL_PATH)
        print(f'✅ SFT 策略模型載入成功: {SFT_MODEL_PATH}')
        
        # 參考模型使用原始基礎模型
        reference_model = deepcopy(base_model)
        print(f'✅ 參考模型準備完成')
        
    except Exception as e:
        print(f'⚠️  SFT 模型載入失敗: {e}')
        print('使用基礎模型進行 DPO 訓練')
        policy_model = base_model
        reference_model = deepcopy(base_model)
else:
    print('⚠️  未找到 SFT 模型，使用基礎模型')
    policy_model = base_model
    reference_model = deepcopy(base_model)

print(f'模型參數量: {policy_model.num_parameters():,}')

## 步驟 3: 準備 DPO 訓練數據

載入偏好數據集並確保格式正確。

In [ ]:
# 載入 DPO 數據集
try:
    dpo_dataset = load_from_disk('./dpo_data')
    print(f'✅ 載入 DPO 數據集，樣本數: {len(dpo_dataset)}')
except:
    print('⚠️  無法載入已保存的數據，創建模擬數據')
    # 創建模擬 DPO 數據
    mock_data = [
        {
            'prompt': '請解釋什麼是機器學習?',
            'chosen': '機器學習是人工智能的一個分支，它讓計算機能夠從數據中學習並做出決策，而無需明確編程。通過算法分析大量數據，系統可以識別模式並提高性能。',
            'rejected': '機器學習就是讓機器變聰明。'
        },
        {
            'prompt': '如何學習程式設計?',
            'chosen': '學習程式設計建議從基礎語法開始，選擇一門適合的語言如Python，多做練習項目，參與開源專案，並持續學習新技術。實作是最重要的學習方式。',
            'rejected': '學程式就是寫code。'
        },
        {
            'prompt': '什麼是深度學習?',
            'chosen': '深度學習是機器學習的一個子領域，使用多層神經網路來學習數據的複雜模式。它在圖像識別、自然語言處理和語音識別等領域取得了突破性進展。',
            'rejected': '深度學習就是很深的學習。'
        }
    ]
    dpo_dataset = Dataset.from_list(mock_data)

print(f'數據集大小: {len(dpo_dataset)}')
print(f'數據欄位: {list(dpo_dataset.features.keys())}')

# 檢查數據格式
sample = dpo_dataset[0]
print(f'\n📝 數據樣本:')
print(f'Prompt: {sample["prompt"]}')
print(f'Chosen: {sample["chosen"][:100]}...')
print(f'Rejected: {sample["rejected"][:100]}...')

## 步驟 4: DPO 損失函數實現

實現 DPO 的核心損失函數，理解其數學原理。

In [ ]:
def compute_log_probs(model, input_ids, attention_mask, labels):
    """計算序列的對數概率"""
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        # 計算每個 token 的對數概率
        log_probs = F.log_softmax(logits, dim=-1)
        
        # 收集目標 token 的對數概率
        target_log_probs = torch.gather(log_probs[:, :-1], dim=-1, 
                                       index=labels[:, 1:].unsqueeze(-1)).squeeze(-1)
        
        # 僅計算非 padding token 的平均對數概率
        mask = (labels[:, 1:] != tokenizer.pad_token_id).float()
        sequence_log_prob = (target_log_probs * mask).sum(dim=1) / mask.sum(dim=1)
        
    return sequence_log_prob


def dpo_loss(policy_chosen_logps, policy_rejected_logps, 
             reference_chosen_logps, reference_rejected_logps, beta=0.1):
    """DPO 損失函數實現
    
    Args:
        policy_chosen_logps: 策略模型對 chosen 回應的對數概率
        policy_rejected_logps: 策略模型對 rejected 回應的對數概率  
        reference_chosen_logps: 參考模型對 chosen 回應的對數概率
        reference_rejected_logps: 參考模型對 rejected 回應的對數概率
        beta: 溫度參數
    """
    # 計算相對於參考模型的獎勵
    chosen_rewards = beta * (policy_chosen_logps - reference_chosen_logps)
    rejected_rewards = beta * (policy_rejected_logps - reference_rejected_logps)
    
    # DPO 損失: 最大化 chosen 相對於 rejected 的偏好
    loss = -F.logsigmoid(chosen_rewards - rejected_rewards).mean()
    
    # 計算準確率 (chosen 獎勵是否高於 rejected)
    accuracy = (chosen_rewards > rejected_rewards).float().mean()
    
    return loss, accuracy, chosen_rewards.mean(), rejected_rewards.mean()


print('✅ DPO 損失函數實現完成')

# 測試損失函數
print('\n🧪 測試 DPO 損失函數:')
dummy_chosen = torch.tensor([-2.0, -1.5])
dummy_rejected = torch.tensor([-3.0, -2.5])
dummy_ref_chosen = torch.tensor([-2.2, -1.7])
dummy_ref_rejected = torch.tensor([-2.8, -2.3])

loss, acc, chosen_reward, rejected_reward = dpo_loss(
    dummy_chosen, dummy_rejected, dummy_ref_chosen, dummy_ref_rejected, beta=0.1
)

print(f'Loss: {loss.item():.4f}')
print(f'Accuracy: {acc.item():.4f}')
print(f'Chosen Reward: {chosen_reward.item():.4f}')
print(f'Rejected Reward: {rejected_reward.item():.4f}')

## 步驟 5: 使用 TRL DPOTrainer

使用 TRL 庫的 DPOTrainer 來簡化 DPO 訓練流程。

In [ ]:
# 準備 DPO 訓練數據格式
def format_dpo_dataset(dataset):
    """將數據集格式化為 DPOTrainer 需要的格式"""
    formatted_data = []
    
    for sample in dataset:
        formatted_data.append({
            'prompt': sample['prompt'],
            'chosen': sample['chosen'],
            'rejected': sample['rejected']
        })
    
    return Dataset.from_list(formatted_data)

# 格式化數據集
formatted_dataset = format_dpo_dataset(dpo_dataset)
print(f'✅ 數據集格式化完成，樣本數: {len(formatted_dataset)}')

# 顯示格式化後的樣本
sample = formatted_dataset[0]
print(f'\n📝 格式化樣本:')
for key, value in sample.items():
    print(f'{key}: {value[:50]}...' if len(str(value)) > 50 else f'{key}: {value}')

## 步驟 6: DPO 訓練配置

設置 DPO 訓練的參數和配置。

In [ ]:
# DPO 訓練配置
output_dir = './dpo_model_output'
os.makedirs(output_dir, exist_ok=True)

# 訓練參數
training_args = TrainingArguments(
    output_dir=output_dir,
    
    # 基本設置
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    learning_rate=1e-6,  # DPO 通常使用較小的學習率
    
    # 優化設置
    warmup_steps=50,
    logging_steps=5,
    save_steps=50,
    
    # 記憶體優化
    fp16=True if torch.cuda.is_available() else False,
    dataloader_pin_memory=False,
    gradient_checkpointing=True,
    
    # 其他設置
    remove_unused_columns=False,
    report_to=None,
    seed=42
)

print('⚙️  DPO 訓練配置:')
print(f'輸出目錄: {output_dir}')
print(f'訓練輪數: {training_args.num_train_epochs}')
print(f'批次大小: {training_args.per_device_train_batch_size}')
print(f'學習率: {training_args.learning_rate}')
print(f'使用 FP16: {training_args.fp16}')

## 步驟 7: 開始 DPO 訓練

使用 DPOTrainer 進行偏好優化訓練。

In [ ]:
try:
    # 創建 DPO Trainer
    dpo_trainer = DPOTrainer(
        model=policy_model,
        ref_model=reference_model,
        args=training_args,
        train_dataset=formatted_dataset,
        tokenizer=tokenizer,
        beta=0.1,  # DPO 溫度參數
        max_length=512,
        max_prompt_length=256,
    )
    
    print('✅ DPOTrainer 創建成功')
    
    # 開始訓練
    print('🚀 開始 DPO 訓練...')
    
    # 訓練前的記憶體狀態
    if torch.cuda.is_available():
        print(f'訓練前 GPU 記憶體: {torch.cuda.memory_allocated() / 1e9:.2f} GB')
    
    # 執行訓練
    dpo_trainer.train()
    
    print('✅ DPO 訓練完成！')
    
    # 訓練後的記憶體狀態
    if torch.cuda.is_available():
        print(f'訓練後 GPU 記憶體: {torch.cuda.memory_allocated() / 1e9:.2f} GB')
        
except Exception as e:
    print(f'❌ DPO 訓練失敗: {e}')
    print('\n🔄 嘗試手動 DPO 訓練實現...')
    
    # 手動 DPO 訓練的簡化版本
    from torch.optim import AdamW
    
    # 準備優化器
    optimizer = AdamW(policy_model.parameters(), lr=1e-6)
    policy_model.train()
    reference_model.eval()
    
    print('🔧 使用手動實現進行 DPO 訓練...')
    
    for epoch in range(1):
        total_loss = 0
        total_acc = 0
        
        for i, sample in enumerate(formatted_dataset):
            # 準備輸入
            prompt = sample['prompt']
            chosen_text = f"{prompt} {sample['chosen']}"
            rejected_text = f"{prompt} {sample['rejected']}"
            
            # Tokenization
            chosen_tokens = tokenizer(chosen_text, return_tensors='pt', truncation=True, max_length=256)
            rejected_tokens = tokenizer(rejected_text, return_tensors='pt', truncation=True, max_length=256)
            
            if torch.cuda.is_available():
                chosen_tokens = {k: v.cuda() for k, v in chosen_tokens.items()}
                rejected_tokens = {k: v.cuda() for k, v in rejected_tokens.items()}
            
            # 計算對數概率
            policy_chosen_logps = compute_log_probs(policy_model, **chosen_tokens, labels=chosen_tokens['input_ids'])
            policy_rejected_logps = compute_log_probs(policy_model, **rejected_tokens, labels=rejected_tokens['input_ids'])
            
            ref_chosen_logps = compute_log_probs(reference_model, **chosen_tokens, labels=chosen_tokens['input_ids'])
            ref_rejected_logps = compute_log_probs(reference_model, **rejected_tokens, labels=rejected_tokens['input_ids'])
            
            # 計算 DPO 損失
            loss, acc, _, _ = dpo_loss(
                policy_chosen_logps, policy_rejected_logps,
                ref_chosen_logps, ref_rejected_logps
            )
            
            # 反向傳播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            total_acc += acc.item()
            
            if i % 2 == 0:
                print(f'Step {i}: Loss={loss.item():.4f}, Acc={acc.item():.4f}')
        
        print(f'Epoch {epoch}: Avg Loss={total_loss/len(formatted_dataset):.4f}, Avg Acc={total_acc/len(formatted_dataset):.4f}')
    
    print('✅ 手動 DPO 訓練完成！')

## 步驟 8: 保存 DPO 模型

保存訓練完成的 DPO 模型。

In [ ]:
# 保存 DPO 模型
print('💾 保存 DPO 模型...')

try:
    # 如果使用 PEFT 模型
    if hasattr(policy_model, 'save_pretrained'):
        policy_model.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)
        print(f'✅ DPO 模型已保存至: {output_dir}')
    else:
        torch.save(policy_model.state_dict(), os.path.join(output_dir, 'dpo_model.pth'))
        tokenizer.save_pretrained(output_dir)
        print(f'✅ DPO 模型狀態已保存至: {output_dir}')
        
except Exception as e:
    print(f'⚠️  模型保存過程中出現問題: {e}')
    print('嘗試基本保存方式...')
    
    # 基本保存方式
    torch.save({
        'model_state_dict': policy_model.state_dict(),
        'config': policy_model.config,
    }, os.path.join(output_dir, 'dpo_model_manual.pth'))
    
    tokenizer.save_pretrained(output_dir)
    print(f'✅ DPO 模型手動保存至: {output_dir}')

# 列出保存的檔案
saved_files = list(Path(output_dir).glob('*'))
print(f'保存的檔案: {[f.name for f in saved_files]}')

## 步驟 9: DPO 模型測試

測試 DPO 訓練後的模型生成能力。

In [ ]:
# 測試 DPO 模型
def test_dpo_model(model, tokenizer, prompt, max_length=150):
    """測試 DPO 模型生成"""
    model.eval()
    
    # 準備輸入
    inputs = tokenizer(prompt, return_tensors='pt')
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}
    
    # 生成回應
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # 解碼輸出
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = generated_text[len(prompt):].strip()
    
    return response

print('🧪 測試 DPO 模型生成能力...')

test_prompts = [
    "Human: 請解釋什麼是機器學習?\n\nAssistant:",
    "Human: 如何學習程式設計?\n\nAssistant:",
    "Human: 什麼是深度學習?\n\nAssistant:"
]

for i, prompt in enumerate(test_prompts):
    print(f'\n📝 測試 {i+1}:')
    print(f'Prompt: {prompt.split("Assistant:")[0]}Assistant:')
    
    try:
        response = test_dpo_model(policy_model, tokenizer, prompt)
        print(f'DPO Generated: {response}')
    except Exception as e:
        print(f'生成失敗: {e}')
    
    print('-' * 80)

## 步驟 10: 訓練總結

總結 DPO 訓練的結果和關鍵指標。

In [ ]:
# DPO 訓練總結
print('=== DPO 訓練總結 ===')
print(f'✅ 基礎模型: {BASE_MODEL_NAME}')
print(f'✅ SFT 模型路徑: {SFT_MODEL_PATH}')
print(f'✅ DPO 訓練數據: {len(formatted_dataset)} 個偏好對')
print(f'✅ 訓練輪數: {training_args.num_train_epochs}')
print(f'✅ 學習率: {training_args.learning_rate}')
print(f'✅ DPO 模型保存位置: {output_dir}')

# 計算模型參數統計
total_params = sum(p.numel() for p in policy_model.parameters())
trainable_params = sum(p.numel() for p in policy_model.parameters() if p.requires_grad)
print(f'✅ 總參數: {total_params:,}')
print(f'✅ 可訓練參數: {trainable_params:,} ({trainable_params/total_params*100:.2f}%)')

print('\n🎯 DPO 對齊模型訓練完成！')
print('下一步: 執行 04-Evaluation_and_Compare.ipynb 進行模型評估')

# 清理 GPU 記憶體
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f'\nGPU 記憶體已清理，當前使用: {torch.cuda.memory_allocated() / 1e9:.2f} GB')

print('\n📚 DPO 核心概念總結:')
print('• DPO 直接從偏好數據中學習，無需獎勵模型')
print('• 使用 Bradley-Terry 模型來建模人類偏好')
print('• 通過對比學習優化模型行為')
print('• 相比 RLHF 更加穩定且易於實現')